# 2 | Fine Tuning In Azure AI Foundry

This section helps you setup the demo for Act 2. You will have done this work a day ahead of the breakout, and have it staged for convenience. 

You can then use the pre-recorded demos we provide or use your staged demo to create a personalized version of those recordings, if preferred. **Just stick to the suggested timings** to ensure you deliver the whole talk.

By the end of this notebook you should have:

1. Setup the Azure AI Foundry project
1. Deployed a base model and validated it with the Playground
1. Setup the `.env` file and configured it to reflect the deployment
1. Got an intuitive sense for the customization journey with
    - Deploying a base model and trying a default user query
    - Prompt engineering it with examples & system message for tone & style
    - Adding data for Retrieval Augmented Generation to get grounded responses
    - Seeing the cost, latency and accuracy achievable without adapting model

This will then set us up for Notebook 2 where we will do base fine-tuning

---

## 2.1 Set The Stage

Before we get to Act 2, we will have shown this slide that features the AI engineer's customization journey. Let's see if we can walk through this with a real project. Here are the main elements of the story we cover here:

1. We have a Zava products database for grounding responses
1. We know the desired "tone & style" we want to achieve
1. Can we get the desired outcome with prompt engineering alone?
1. Can we improve the outcome if we add data with RAG?
1. Why should we continue to the customization step?

_Note:_ The image shows a "sample" query-response. In our demo we may use a different one but we want to emphasize the model behavior change we want to see:

1. Cora is a polite, factual and helpful assistant helping customers find Zava products.
1. (Polite) Cora begins by acknowledging user query ("Good Choice!")
1. (Factual) Cora is factual by providing grounded response ("I... Eggshell Paint")
1. (Helpful) Cora is helpful by offering to follow-up ("Would you like to know more ...")

![story](./06.png)

---

## 2.2 Understand Zava Data

Zava is a retailer for home improvement products. In the `data/zava/` folder you fill find the following files:

1. `products.json` - a JSON formatted product catalog with 420+ items
1. `products.csv` - a CSV version of the same file
1. `products-paints.csv` - the subset with a `PAINT & FINISHES` category

To get this subset, we just filtered the file for lines with that category using GitHub Copilot (Agent Mode) for fast completions. This is what a sample product item in this category looks like. We do _NOT_ have the imags data locally, so we will focus instead of Q&A around other attributes of these products.

In [10]:
import pandas as pd

# Read the CSV file
products_df = pd.read_csv('../../../data/zava/products-paints.csv')

# Display the first two products in a readable format (excluding image_path)
columns_to_show = ['name', 'sku', 'price', 'description', 'stock_level', 'main_category', 'subcategory']

# Pretty print the first 2 products for example
for i in range(min(2, len(products_df))):
    print(f"=== Product {i+1} ===")
    for col in columns_to_show:
        print(f"{col.replace('_', ' ').title()}: {products_df.iloc[i][col]}")
    print()  # Empty line between products

=== Product 1 ===
Name: Premium Interior Latex Flat
Sku: PFIP000001
Price: 40.0
Description: High-quality flat interior paint with excellent coverage and hide, perfect for ceilings and low-traffic areas.
Stock Level: 19
Main Category: PAINT & FINISHES
Subcategory: INTERIOR PAINT

=== Product 2 ===
Name: Interior Eggshell Paint
Sku: PFIP000002
Price: 44.0
Description: Durable eggshell finish paint with subtle sheen, ideal for living rooms and bedrooms with easy cleanup.
Stock Level: 80
Main Category: PAINT & FINISHES
Subcategory: INTERIOR PAINT



---

## 2.2 Create Sample Q&A Data

Let's get a few examples of question-answer pairs that reflect our **desired tone and style** for Cora. For convenience, we just generated these using GitHub Copilot in Agent Mode, studying the `products-paints.csv` file with this guidance:

**You may need to revise the guidance interactively**

```txt
Study the `data/zava/products-paints.csv` file to understand the product catalog details. We are generating sample customer support question-answer pairs in a JSONL format for fine-tuning.

Here is an example of a valid question-answer pair:
{
  "question": "Do you have any paint that works well in bathrooms?",
  "answer": "Yes! Our Interior Semi-Gloss Paint has moisture resistance, perfect for bathrooms. Need more details?"
}

Use this as a template to generate 10 additional question-answer pairs in a file called `qa-fewshot.jsonl` in the data/zava folder where:
1. Each response starts with a brief and friendly acknowledgement (for example: "Yes!" or "Of course!")
2. Response then provides factual answer using data in the products-paints.csv file
3. Response ends with offer to help do a follow-up task related to that response

Avoid duplicate questions. 
Refine answers to be less chatty and more actionable
Have questions on different aspects of product (price, stock etc.)
Have responses that reflect different kinds of customer support behaviors (not in stock, currently on sale, the requested product will not meet the stated user requirement )

```

In [15]:
import json

# Read the JSONL file and pretty print the first 3 question-answer pairs
jsonl_file_path = '../../../data/zava/qa-fewshot.jsonl'

# Generated pairs
print(f"Total Q&A pairs created: {sum(1 for line in open(jsonl_file_path))}\n")

# Pretty print the first 3 Q&A pairs for example
with open(jsonl_file_path, 'r') as file:
    for i, line in enumerate(file):
        if i >= 3:  # Only show first 3
            break
        
        qa_pair = json.loads(line.strip())
        
        print(f"--- Example {i+1} ---")
        print(f"Question: {qa_pair['question']}")
        print(f"Answer: {qa_pair['answer']}")
        print()  # Empty line between pairs


Total Q&A pairs created: 10

--- Example 1 ---
Question: Do you have any paint that works well in bathrooms?
Answer: Yes! Our Interior Semi-Gloss Paint has moisture resistance, perfect for bathrooms. Need details?

--- Example 2 ---
Question: What's your most affordable interior paint option?
Answer: I can help! Our Drywall Primer is $29, great for new drywall. Want a topcoat too?

--- Example 3 ---
Question: I need paint for my deck - what do you recommend?
Answer: Absolutely! Deck and Fence Stain at $42 with UV protection. Check stock levels?



---

## 2.3 Create Foundry Project & Resource

This helps organize your work for a specific use case or AI solution. 
There are [two types of projects](https://ai.azure.com/doc/azure/ai-foundry/how-to/migrate-project?) you can create - Hub-based (for machine learning studio use) and Foundry-based (for generative and agentic AI). **Use Foundry Projects** to get streamlined setup, workflow and governance for a _platform-as-a-service_ approach (unified SDK, API) - recommended for new projects!

![Project types](https://learn.microsoft.com/azure/ai-foundry/media/migrate-project/project-structure.svg)


Let's create the project now.

1. Visit [https://ai.azure.com/allResources](https://ai.azure.com/allResources) 
1. Log in with Azure credentials to view resources.
1. Click **Create New** and [follow instructions](https://ai.azure.com/doc/azure/ai-foundry/how-to/create-projects) to create a Foundry project.
1. Completes in minutes. **You see a Project overview page**.

Note these features on that page - we'll revisit them later:
1. Overview page - has endpoints and keys
1. Sidebar menu - has Model catalog & Playgrounds options
1. Sidebar menu - has "My Assets" section with Models & Endpoints
1. Sidebar menu - has "Management Center" for resource management

---

## 2.4 Deploy Base Model & Iterate

To get started on a model customization journey, the first step is to **select a base model** for our Cora chatbot. Let's start by deploying a popular general-purpose large language model (`gpt-4.1`).

We can then see if it meets our needs out of the box with a simple system message and test prompt.

### 2.4.1 Deploy GPT-4.1

1. Return to the "Project Overview" tab in Azure AI Foundry portal
1. Click on the "Model catalog" option (left sidebar) in the project overview.
1. Search for `gpt-4.1` model - click "Use this model" and complete deployment
1. You should see the **Model Details** card.
1. Click **Open in playground** to validate it interactively.

You should see something like this:

![GPT4 Playground](./00-playground-gpt41.png)

### 2.4.2 Set System Message

The Playground provides these elements for customization:
1. **System Prompt** - Give model instructions & context for request. 
1. **Add Section** - drop down (examples, variables, safety system messages)
1. **Add Your Data** - launch wizard to connect to external data sources
1. **Parameters** - max completion tokens, temperature, top-p etc.

Copy this to System Message window and click "Apply" to enforce it.

> You are Cora - a polite, factual and helpful assistant for Zava customers.

Now, try this sample prompt to see response _with default base model_.

> Do you have any paint that works well in bathrooms?


**WHAT WE SEE**

1. Response is not grounded (it fabricated Zava products)
1. Tone & style are not what we want (very verbose)
1. Response was fairly quick but used 175 tokens

![Response](./01-prompt-gpt41.png)

**DESIRED TONE & STYLE**

> Yes! Our Interior Semi-Gloss Paint has moisture resistance, perfect for bathrooms. Need details?"

### 2.4.3 Add Instructions & Examples

Use the **Add section** to add examples. You will be asked to enter "User" and "Assistant" sections for each example. We'll use the Q&A samples we created where the Question goes into User dialog and Answer goes into Assistant dialog. For now, let's just add 5 examples reflecting different situations.


User: "Do you have any paint that works well in bathrooms?"

Assistant: "Yes! Our Interior Semi-Gloss Paint has moisture resistance, perfect for bathrooms. Need details?"

User: "What's your most affordable interior paint option?"

Assistant: "I can help! Our Drywall Primer is $29, great for new drywall. Want a topcoat too?"

### 2.4.4 Test Prompt-Engineered Tone

You can now use one of our sample questions (that was not in the example) and see if the response is **similar** to what we expected

| Sample | Sample Answer |
|:---|:---|
| "I need paint for my deck - what do you recommend?" | "Absolutely! Deck and Fence Stain at $42 with UV protection. Check stock levels?"|
| "Do you have any eco-friendly paint options?" | "Yes! Zero VOC Interior Paint at $52 ensures healthy air quality. Need color options?" |
| "I'm looking for paint tray liners - are they in stock?" | "Unfortunately, Paint Tray Liner Set is out of stock. Try our Disposable Paint Tray Set instead?" |

Let's see what we actually get.

**Answer 1**: Using few-shot examples we were able to get the tone and style _closer_ to what we need. 

But note:
1. The answer is not factual. This product does not exist.
1. The style is closer to what we want - but it is a bit longer than what we suggest in our examples. We can improve it.
1. Every prompt will now add 63 tokens to carry those examples to "prompt engineer" responses effectively. 
1. If we want to have more diverse examples (e.g., apology tone for out of stock etc.) this length will increase.
1. We have a fixed context window so we can only add a limited number of examples 

![Example 1](./02-fewshot-gpt41-1.png)


**Answer 2** - 

1. This is closer in style with a shorter response - but it is not factual. 
1. Few shot examples are taking up prompt length space again.
1. The style is not perfect - note how our example responses always cited prices but model does not

![Example 2](./02-fewshot-gpt41-2.png)


## 2.5 Add Your Data (For RAG)

For the next step of customization, we'll use RAG to ground responses in our product data. To do this we need to take 3 steps:

1. Setup environment variables - for code-first use
1. Authenticate with Azure - to use managed identity
1. Create an Azure AI Search resource - and add connection to Foundry project
1. Run scripts - to update RBAC roles and create index from data
1. Use created index in playground - verify we get grounded responses

**These steps are being done manually for simplicity. We may replace them later with an AZD template or Azure CLI scripts for convenience, once those are tested and validated**

### 2.5.1 Authenticate With Azure 

Use `az login --tenant <tenant-id>` to authenticate with Azure from the VS Code terminal. This process will trigger a workflow that will complete authentication using a device code in the browser. Once completed, return here and run the next cell to verify you are authenticated

**Logging in allows us to use Managed Identity later for keyless authentication (recommended) in code-first examples later**

In [ ]:
# Check if you are logged into Azure 
!az account show

# If you are not logged in you will see a message like 
#    `Please run 'az login' to setup account.`

### 2.5.2 Create `.env` file

1. Copy over `.env.sample` to `.env` - we'll fill in various values here as we go
1. Start by completing the first section for your Azure AI Foundry resource.
    - AZURE_OPENAI_API_KEY - from the Azure AI Foundry Overview page 
    - AZURE_OPENAI_ENDPOINT - from the Azure AI Foundry Overview page 
    - AZURE_OPENAI_API_VERSION - leave default value of "2025-02-01-preview"  
1. Start filling in the Azure AI Search section
    - AZURE_AISEARCH_ENDPOINT - **we will do this in the next section**
    - AZURE_AISEARCH_INDEX - set this to "zava-products"
    - AZURE_AISEARCH_RESOURCE_GROUP - from the Azure AI Foundry Overview page (right)
```

### 2.5.3 Create Azure AI Search Resource

We want to make our data accessible to the model for RAG. Use [this guidance](https://learn.microsoft.com/azure/ai-foundry/tutorials/copilot-sdk-create-resources#create-search) to get this done, next.

1. Visit your AI Project Overview page - look for the **Resource Group** link (panel, right)
1. Click to visit Azure Portal - use [this link](https://portal.azure.com/#create/Microsoft.Search) to create AI Search resource **for that resource group**
1. Use a relevant name and default location - **make sure you pick Standard Pricing Tier**
1. Click **Review+Create** to complete -- this will setup Azure AI Search resource in the same resource group
1. Wait till done - click **Go to deployed resource** to see the Azure AI Search resource
1. Look for the `Url` link in essentials - **use it to update .env value for AZURE_AISEARCH_ENDPOINT**
1. Visit the `Settings > Keys` blade for Azure AI Search - switch API Access Control to "Both"

This last step is needed in order to allow Playground to access index seamlessly. Our local development uses only managed identity so we don't need to store the key in our environment variables.

_Note: This tier has a non-trivial cost per month. Make sure you tear this down as soon as your breakout session is over_.

### 2.5.4 Create Azure AI Search Index

Before we can run the notebook that creates the index, we need to make sure our user identity has the right RBAC permissions.

1. Open a VS Code Terminal - make sure you are in the root directory
1. Run this command `./data/scripts/01-update-roles.sh` - takes a minute or two to complete
1. You should now be set to run the notebook that creates the index!!

We also need to make sure we have the `text-embedding-ada-002` model deployed in our Azure AI Foundry project to help create the vector search indexesx. 

1. Return to Azure AI Foundry - click your project Overview
1. Go to Model Catalog - search for `text-embedding-ada-002` and deploy it.
1. It should compelete within minutes - verify the `Models + endpoints` now shows the model.

Return to the VS Code editor and open the `data/zava/02-create-aisearch-index.ipynb` notebook

1. Select Kernel - default 3.12 is fine
1. Run All - all cells in notebook should complete successfully
1. The last cell output should show something like this

    ```txt
    Uploading 51 Zava products to index zava-products
    Successfully uploaded 51 products to the search index!
    The Zava product catalog is now ready for semantic search.
    ```

**Let's verify the index, next!**

### 2.5.5 Verify Azure AI Search Index

1. Visit the Azure Portal Azure AI Search resource - click "Search Explorer".
1. You should see a Search Explorer with "zava-products" as an indext
1. Enter a sample user query - e.g., `Do you have any paint that works well in bathrooms?`
1. Note that the _retrieved result_ contains "Interior Semi-Gloss Paint" - the right product!

![Index](./03-search-index.png)

### 2.5.6 Use Index In Playground

1. Return to the Playground where we were testing the base model.
1. Now click "Add Data" - then click **Add Data Source** 
1. Select "Azure AI Search" - the wizard will help you complete the flow
1. You should see "zava-products" as an index option - select it
1. Select the "Add vector search" option - use pre-deployed `text-embedding-ada-002` model 
1. (Optional) select custom field mapping - view default mappings (leave them as is)
1. Data management - leave defaults (Hybrid + semantic search)
1. Authentication - select API key for now (will need to add AD roles support later)

![Add data](./04-add-data-source.png)

To use Managed Identity for Authentication, we need to allow services [to authorize each other](https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/on-your-data-configuration#role-assignments). This requires a few additional steps that we will fix later
**`TODO`** - update this step for managed identity.

For now, we'll move ahead by allowing API KEY usage instead.


### 2.5.7 Test Prompt with Data

Return to the Playground. You will now see the Azure AI Search index is a data source, your system message provides guidance and you have two examples for few-shot training. Try the same request as before and you will see something like this. _Note: Since LLMs are stochastic, your exact response may vary_.

Note that now we get a response that is _factually correct_ - it identifies a valid product in the Zava database that is relevant to the user query. However, it may not be the _precise_ response we were looking for. To test this out, visit the Azure AI Search resource "explorer" and run these two queries.

1. "Deck stain" => top answer "Deck and Fence Stain" - _This is the answer we want_
1. "Paint deck" => top answer "Solid Color Deck Stain" - _This is the answer we get_.

![Grounded](./05-withrag-gpt41.png)


We can improve this with an enhanced prompt template and orchestration framework etc. - but for now, we just wanted to demonstrate three things:

1. Base models will not reflect a desired tone, style - or be grounded in your data
1. Prompt engineering (with system message and few show examples) - can improve tone/style 
1. Adding data (RAG with vector search) - can improve groundedness in your data
1. Both these "customization" steps add costs (prompt length, network latency, context window usage)
1. To improve performance, we may need more data, more 

But each step adds a certain cost/accuracy tradeoff
1. Improve tone with more few shot examples - but this increases prompt length
1. Improve groundedness by retrieving more examples - but results are still not precise
1. Few shot examples are limited - having more examples would help improve tone and style
1. RAG retrieves grounded results - improving how they are used by model would improve precision

**How can we customize the model further to meet these targets?** This is where fine-tuning can help.

![More RAG](./06-morerag-gpt41.png)


